<a href="https://colab.research.google.com/github/And0k/jupyter-space/blob/main/%D0%9A%D0%BE%D1%80%D0%B6_%D0%90_%D0%9C%D0%98%D0%98_%D0%A0%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D1%81%D1%8A%D0%B5%D0%B4%D0%BE%D0%B1%D0%BD%D1%8B%D1%85_%D0%B4%D0%B8%D0%BA%D0%BE%D1%80%D0%B0%D1%81%D1%82%D1%83%D1%89%D0%B8%D1%85_%D1%80%D0%B0%D1%81%D1%82%D0%B5%D0%BD%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os

In [ ]:
import plotly.express as px  # Library that generates interactive web-based visualizations.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
dataset_name = "gverzea/edible-wild-plants"
data_file_name = f"{dataset_name.split('/', 1)[-1]}.zip"  # file name that appears to be downloaded after downloading dataset
my_drive_dest = "drive/My Drive/Colab Notebooks/Методы ИИ"
b_data_exist = os.path.exists(rf'{my_drive_dest}/{data_file_name}')
b_data_exist

True

# Загрузка с Kaggle и копирование на Google Диск (при первом запуске)
(you'll need press button to select and upload kaggle.json from you local computer if interaction with Kaggle will be needed)

In [ ]:
if not b_data_exist:
    my_drive_dest_for_cmd = my_drive_dest.replace(' ', '\ ')

    # Kaggle access file (from my_drive_dest or upload)
    if not os.path.exists(os.path.expanduser('~/.kaggle/kaggle.json')):
        !mkdir ~/.kaggle
        if os.path.exists(rf'{my_drive_dest}/kaggle.json'):
            !cp $my_drive_dest_for_cmd/kaggle.json ~/.kaggle/
        else:
            print("Select kaggle.json to dounload to Google Colab")
            from google.colab import files
            uploaded = files.upload()
            !cp kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json

    # Dataset zip file (autoskipping if exist)
    !kaggle datasets download $dataset_name -p $my_drive_dest_for_cmd


# # Directory to extract the contents
# extract_path = 'dataset_folder'

# # Create the extraction directory if it doesn't exist
# os.makedirs(extract_path, exist_ok=True)

# # Extract the zip file
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# Просмотр состава данных

In [ ]:
df_in = pd.read_csv(
    rf'{my_drive_dest}/{data_file_name}', parse_dates=['Publication date']
)
df_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867 entries, 0 to 866
Data columns (total 38 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   System                              867 non-null    object        
 1   Domain                              867 non-null    object        
 2   Organization                        853 non-null    object        
 3   Organization categorization         853 non-null    object        
 4   Country (from Organization)         853 non-null    object        
 5   Authors                             836 non-null    object        
 6   Publication date                    867 non-null    datetime64[ns]
 7   Reference                           858 non-null    object        
 8   Link                                865 non-null    object        
 9   Citations                           758 non-null    float64       
 10  Notability criteria       

## Ненужный для данной работы шаг

In [ ]:
# Convert object columns with low cardinality and repetitive values
# with number of unique values < 5% to categorical data type
good_categorical_cols = (
    (df_in.dtypes == 'object') &
    (df_in.nunique() < df_in.shape[0] * 0.05)
)
df_in.loc[:, good_categorical_cols.values] = df_in.loc[:, good_categorical_cols.values].astype('category')

# Try auto find suitable columns for better type and convert them
df_in = df_in.convert_dtypes()  # not converts to categorical

df_in.info()
display(df_in.head())
display(df_in.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867 entries, 0 to 866
Data columns (total 38 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   System                              867 non-null    string        
 1   Domain                              867 non-null    string        
 2   Organization                        853 non-null    string        
 3   Organization categorization         853 non-null    string        
 4   Country (from Organization)         853 non-null    string        
 5   Authors                             836 non-null    string        
 6   Publication date                    867 non-null    datetime64[ns]
 7   Reference                           858 non-null    string        
 8   Link                                865 non-null    string        
 9   Citations                           758 non-null    Int64         
 10  Notability criteria       

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1057: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1081: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


,System,Domain,Organization,Organization categorization,Country (from Organization),Authors,Publication date,Reference,Link,Citations,...,Confidence,Abstract,Model accessibility,Base model,Finetune compute (FLOP),Finetune compute notes,Batch size,Batch size notes,Frontier model,Training power draw (W)
0,Qwen2.5-72B,Language,Alibaba,Industry,China,<NA>,2024-09-19,Qwen2.5: A Party of Foundation Models!,https://qwenlm.github.io/blog/qwen2.5/,<NA>,...,Confident,In the past three months since Qwen2’s release...,Open access (unrestricted),<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,Table Tennis Agent,Robotics,Google DeepMind,Industry,Multinational,"David B. D'Ambrosio, Saminda Abeyruwan, Laura ...",2024-08-07,Achieving Human Level Competitive Robot Table ...,https://deepmind.google/research/publications/...,<NA>,...,Likely,Achieving human-level speed and performance on...,Unreleased,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,AFM-server,Language,Apple,Industry,United States of America,"Andy Narayanan, Aonan Zhang, Bowen Zhang, Chen...",2024-07-29,Apple Intelligence Foundation Language Models,https://machinelearning.apple.com/research/app...,<NA>,...,Confident,<NA>,Hosted access (no API),<NA>,<NA>,<NA>,18949752.575891,Main pretraining uses sequence length of 4096 ...,<NA>,3466813.588174
3,AFM-on-device,Language,Apple,Industry,United States of America,"Andy Narayanan, Aonan Zhang, Bowen Zhang, Chen...",2024-07-29,Apple Intelligence Foundation Language Models,https://machinelearning.apple.com/research/app...,<NA>,...,Confident,We present foundation language models develope...,Hosted access (no API),<NA>,<NA>,<NA>,18949752.575891,Main pretraining uses sequence length of 4096 ...,<NA>,<NA>
4,Mistral Large 2,Language,Mistral AI,Industry,France,"Albert Jiang, Alexandre Sablayrolles, Alexis T...",2024-07-24,"Top-tier reasoning for high-complexity tasks, ...",https://mistral.ai/news/mistral-large-2407/,<NA>,...,Likely,"Today, we are announcing Mistral Large 2, the ...",Open access (non-commercial),<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


,System,Domain,Organization,Organization categorization,Country (from Organization),Authors,Publication date,Reference,Link,Citations,...,Confidence,Abstract,Model accessibility,Base model,Finetune compute (FLOP),Finetune compute notes,Batch size,Batch size notes,Frontier model,Training power draw (W)
862,Sequence-based pattern recognition,Vision,Massachusetts Institute of Technology (MIT),Academia,United States of America,O. G. Selfridge,1955-03-01,Pattern recognition and modern computers,https://dl.acm.org/doi/10.1145/1455292.1455310,290,...,Unknown,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
863,Self Organizing System,Other,Massachusetts Institute of Technology (MIT),Academia,United States of America,W. A. Clark and B. G. Farley,1955-03-01,Generalization of pattern recognition in a sel...,https://dl.acm.org/doi/10.1145/1455292.1455309,93,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
864,Genetic algorithm,Mathematics,Institute for Advanced Study,Academia,United States of America,NA Barricelli,1954-07-02,Numerical testing of evolution theories,https://link.springer.com/article/10.1007/BF01...,266,...,Unknown,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
865,SNARC,Robotics,Harvard University,Academia,United States of America,Marvin Minsky,1952-01-08,A Neural-Analogue Calculator Based upon a Prob...,https://en.wikipedia.org/wiki/Stochastic_neura...,33,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
866,Theseus,Robotics,Bell Laboratories,Industry,United States of America,Claude Shannon,1950-07-02,Mighty Mouse,https://www.technologyreview.com/2018/12/19/13...,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True,<NA>


## Расчет матрицы корреляций по Пирсонy
### Используем только типы данных, которые подходят для расчета корреляции Пирсона

In [ ]:
df = df_in.select_dtypes(include=['number', 'datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867 entries, 0 to 866
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Publication date                    867 non-null    datetime64[ns]
 1   Citations                           758 non-null    Int64         
 2   Parameters                          580 non-null    Float64       
 3   Training compute (FLOP)             422 non-null    Float64       
 4   Training dataset size (datapoints)  429 non-null    Float64       
 5   Epochs                              210 non-null    Float64       
 6   Training time (hours)               191 non-null    Float64       
 7   Hardware quantity                   156 non-null    Int64         
 8   Hardware utilization                32 non-null     Float64       
 9   Training compute cost (2023 USD)    153 non-null    Float64       
 10  Finetune compute (FLOP)   

In [ ]:
df_corr =  df.corr(min_periods=10)
#Plotting Visualizing heatmap correlation.
Heatmap = px.imshow(df_corr, text_auto=True,  color_continuous_scale='Viridis', aspect='auto')
#Naming a title, setting figure size, and background color.
Heatmap.update_layout(
     title={
        'text': 'Correlation Heatmap of AI Models(only with Numeric Features)',
        'x': 0.5,  #Centering the title.
        'xanchor': 'center',  #Ensure the title is centered.
        'yanchor': 'top'
     },
     paper_bgcolor='black', #For background color outside the plot area.
     plot_bgcolor='black',  #For background color of the plot area.
     font_color='white', #Changing the font color to white for readability and visibility.
     height=600    #Height in pixels.
)

#Displaying the heatmap.
Heatmap.show()

# Информация о датасете из др. источников
- В наборе данных нет информации чтоб охарактеризовать многомодельные модели
- Колонка Confidence характеризует модели, состав которых вероятно является коммерческой тайной

In [ ]:
# Графики от времени публикации

In [ ]:
# Кол-во параметров (размеры) моделей
express.scatter(data_frame=df, x='Publication date', y='Parameters', hover_name='System', color='Domain', height=800, log_y=True)

In [ ]:
# количество цитирований
express.scatter(data_frame=df, x='Publication date', y='Citations', hover_name='System', log_y=True)

Видно, что со временем
- модели становятся больше
- число цитирований растет, но не у всех моделей


In [ ]:
df.describe()

FileNotFoundError: [Errno 2] No such file or directory: 'notable_ai_models.csv'